In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import pynvml
import psutil
import string

def remove_non_printable(s):
    return ''.join(c for c in s if c not in string.printable)

/home/jiaoyilin/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#查看内存的函数
def get_avaliable_memory(device):
    if device==torch.device('cuda:0'):
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)        # 0表示第一块显卡
        meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
        ava_mem=round(meminfo.free/1024**2)
        print('current available video memory is' +' : '+ str(round(meminfo.free/1024**2)) +' MIB')

    elif device==torch.device('cpu'):
        mem = psutil.virtual_memory()
        print('current available memory is' +' : '+ str(round(mem.used/1024**2)) +' MIB')
        ava_mem=round(mem.used/1024**2)

    return ava_mem


In [2]:
#MD换成97终于能做了，如果CPU和内存都好一些，可以尝试用320M的
def data_dealing(file_path):
    df = pd.read_excel(file_path)
    tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-97M-Chinese')    #这个写作的话需要引用别人论文

    features=[]
    for i in range(len(df)):
        text=df['回复内容'][i]
        new_text=remove_non_printable(text) #去除大部分文本占位符
        input_id = tokenizer.encode(new_text)
        if len(input_id)>1500:
            print(i,text,df['Unnamed: 0'][i])
            continue
        #print(i,df['政府回复类型'][i])     #测试时候用
        answer = df['政府回复类型'][i]
        if answer=='解决':                  #知悉是0，解决是1，解释是2
            tag=1
        if answer=='解释':
            tag=2
        if answer=='知悉':
            tag=0
        feature = {'input_id':input_id,'tags':tag,'input_len':len(input_id)}   #记录下文本内容吧，别很多超过510的
        features.append(feature)
    return features

In [6]:
#MD换成97终于能做了，如果CPU和内存都好一些，可以尝试用320M的
def data_deal_liuyan(file_path):
    df = pd.read_excel(file_path)
    tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-320M-Chinese')    #这个写作的话需要引用别人论文

    features=[]
    for i in range(len(df)):
        text=df['留言文本'][i]
        #print(i,df['政府回复类型'][i])     #测试时候用
        new_text=remove_non_printable(text)
        input_id = tokenizer.encode(new_text)
        
        answer = df['投诉类型'][i]
        if answer==0:                  #抑制是0，建设是1，中立是2
            tag=0
        if answer==1:
            tag=1
        if answer=='中立':
            tag=2 
        feature = {'input_id':input_id,'tags':tag,'input_len':len(input_id)}   #记录下文本内容吧，别很多超过510的
        features.append(feature)
    return features

In [2]:
#这里是因为要用已有的模型来标记数据，所以新设置一个
def test_collate_fn(batch):
    max_len = max([len(f["input_id"]) for f in batch])
    input_id = [f['input_id'] + [0] * (max_len - len(f['input_id'])) for f in batch]
    input_mask = [[1] * len(f['input_id']) + [0] * (max_len - len(f['input_id'])) for f in batch]
        
    input_id = torch.tensor(input_id)
    input_mask = torch.tensor(input_mask)
    
    output = (input_id, input_mask)
    return output

In [5]:
#这里换个思路，每跑一次就存一次！
def add_data(file_path,model,file_name,device):
    df = pd.read_excel(file_path)       #光这一步就要占用很多内存
    tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-320M-Chinese')    #这个写作的话需要引用别人论文
        #先获得数据
    features=[]
    for i in range(len(df)):
        text=df['leaveContent'][i]
        if pd.isna(text) or type(text)==int:
            continue                          #如果是nan或者文本是数字就跳过
        new_text=remove_non_printable(text)   #去除大部分转义字符
        if len(new_text)>2000:                #有7000多字的回复你敢信？设置3000的时候还是会爆内存，就干脆设置2000吧
            continue
        input_id = tokenizer.encode(new_text)  
        feature = {'input_id':input_id,'index':i,}  
        features.append(feature)              
    print('数据总量:',len(df),'feature数量:',len(features))
    
    preds=[]
    dataloader =  DataLoader(features, batch_size=2, shuffle=False, collate_fn=test_collate_fn, drop_last=False)   
    for step, batch in enumerate(dataloader):
        model.eval()
        inputs = {'input_id': batch[0].to(device),
                  'input_mask': batch[1].to(device),
              }
        with torch.no_grad():
            output = model(**inputs)     
            preds.extend(output[0].cpu())       
            torch.cuda.empty_cache()           #加上这一步主动释放内存就ok了
        if step%5000==0:
            print("当前运行数据:",step*2) 
            
    result=[]                    #知悉是0，解决是1，解释是2
    for n in range(len(preds)):
        y_pred=preds[n].argmax(dim=-1)
        if y_pred==0:
            result.append('抑制')
        if y_pred==1:
            result.append('建设')
        if y_pred==2:
            result.append('中立')
        
    #这里注意一下，这个df的数据类型，需要直接放入长度相同的list才能创建，比较麻烦
    new_result=[]
    index=0       
    for i in range(len(df)):
        if index==len(features) or i!=features[index]['index']:
            new_result.append('留言内容过长')
        else:
            new_result.append(result[index])
            index += 1
    df['留言类型（机器）']=new_result
    df.to_excel('final_result/'+file_name,sheet_name='sheet1',index=False)
    
    return new_result,preds

In [ ]:

#11到18这几年的留言回复花了6个h，19到22得5个小时，夸张！
#然后这里还是用liuyan_result的数据做结果，只不过最后读数据读新result的数据就行

%%time
x=list(range(2021,2023))   #19年的跑完了
for num in x:
    file_path='final_result/'+str(num)+'.xlsx'
    file_name=str(num)+'_new.xlsx'
    print(file_path,file_name)
    y=add_data(file_path,my_model,file_name,device)
#然后留言是19到22都没做，直接从final_result里的19到22做就行

file_path='留言板/2019上.xlsx'
df = pd.read_excel(file_path)
df.head()

#先看下数据能否读的进去
df = pd.read_excel('data.xlsx')
df.head()

#先测试下分词效果
tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-710M-Chinese')    #这个写作的话需要引用别人论文
test_text=df['回复内容'][2]
#print(type(test_text))
#print(test_text)
x=tokenizer.encode(test_text)
len(x)
#从结果来看，就是单纯的把字分开了，不是纯粹的分词，这个不知道会不会对性能有影响，先做着看
#x=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_text))
#记得加CLS和SEP，测试通过